In [ ]:
import json

from concurrent.futures import ThreadPoolExecutor

import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
from web3 import Web3

from mainnet_launch.database.schema.full import RebalancePlans, Destinations, DexSwapSteps, Tokens, RebalanceEvents

from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_orm,
    get_full_table_as_df,
    insert_avoid_conflicts,
    get_subset_not_already_in_column,
)

from mainnet_launch.data_fetching.tokemak_subgraph import fetch_autopool_rebalance_events_from_subgraph
from mainnet_launch.constants import ALL_AUTOPOOLS, AutopoolConstants, USDC, WETH, AUTO_ETH


autopool = AUTO_ETH

s3_client = boto3.client("s3", config=Config(signature_version=UNSIGNED))
rebalance_plan_df: list[RebalancePlans] = get_full_table_as_df(
    RebalancePlans, where_clause=(RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr)
)
rebalance_event_df = fetch_autopool_rebalance_events_from_subgraph(autopool)

rebalance_event_df["datetime_executed"] = pd.to_datetime(
    rebalance_event_df["timestamp"].astype(int), unit="s", utc=True
)
rebalance_event_df


def _connect_plans_to_rebalance_evnets(rebalance_event_df: pd.DataFrame, rebalance_plan_df: pd.DataFrame) -> dict:
    rebalance_transaction_hash_to_rebalance_plan = {}

    for index in range(len(rebalance_event_df)):
        one_rebalance_event = rebalance_event_df.iloc[index]

        tx_hash = one_rebalance_event["transactionHash"]
        same_destinations = (rebalance_plan_df["token_out"] == one_rebalance_event["tokenOutAddress"]) & (
            rebalance_plan_df["token_in"] == one_rebalance_event["tokenInAddress"]
        )
        same_amount_out = rebalance_plan_df["amount_out"] == one_rebalance_event["tokenOutAmount"]

        window_start = one_rebalance_event["datetime_executed"] - pd.Timedelta(minutes=10)

        generated_no_more_than_one_hour_before = rebalance_plan_df["datetime_generated"].between(
            window_start, one_rebalance_event["datetime_executed"]
        )

        matches = rebalance_plan_df[same_destinations & same_amount_out & generated_no_more_than_one_hour_before]
        matches = matches.sort_values("datetime_generated", ascending=False).head(1)

        if len(matches) == 0:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = None
        else:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = matches["file_name"].values[0]

    return rebalance_transaction_hash_to_rebalance_plan


rebalance_transaction_hash_to_rebalance_plan = _connect_plans_to_rebalance_evnets(rebalance_event_df, rebalance_plan_df)

rebalance_event_df["rebalance_file_path"] = rebalance_event_df["transactionHash"].map(
    rebalance_transaction_hash_to_rebalance_plan
)
rebalance_event_df["autopool_vault_address"] = autopool.autopool_eth_addr
rebalance_event_df["chain_id"] = autopool.chain.chain_id


def _extract_rebalance_event_row(row: pd.Series) -> RebalanceEvents:
    return RebalanceEvents(
        tx_hash=row["transactionHash"],
        autopool_vault_address=row["autopool_vault_address"],
        chain_id=row["chain_id"],
        rebalance_file_path=row["rebalance_file_path"],
        destination_out=row["destinationOutAddress"],
        destination_in=row["destinationInAddress"],
        quantity_out=row["tokenOutAmount"],
        quantity_in=row["tokenInAmount"],
    )


new_rebalance_events_rows = rebalance_event_df.apply(_extract_rebalance_event_row, axis=1).to_list()

new_rebalance_events_rows

2025-05-28 15:43:29.604 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2025-05-28 15:43:30,528 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-28 15:43:30,528 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-28 15:43:30,718 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-28 15:43:30,718 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-28 15:43:30,900 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-28 15:43:30,900 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-28 15:43:31,088 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-28 15:43:31,089 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

,transactionHash,timestamp,blockNumber,tokenInAmount,tokenIn,tokenOutAmount,tokenOut,destinationInAddress,destinationOutAddress,tokenInAddress,tokenOutAddress,datetime_executed
20,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,1726602143,20772483,99.961014,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,100.000000,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-17 19:42:23+00:00
6,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,1726632143,20774967,314.380624,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,314.500371,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 04:02:23+00:00
162,0xaa013ce865573532459a55487fb809c3e93edeed05af...,1726676555,20778643,144.676019,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,146.034135,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-18 16:22:35+00:00
405,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,1726714667,20781802,133.437132,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,134.705433,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 02:57:47+00:00
13,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,1726752383,20784928,335.615197,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,335.831303,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xE382BBd32C4E202185762eA433278f4ED9E6151E,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2024-09-19 13:26:23+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
261,0x7356f89ae7b56795daec5ca6476ee2ce97f8124b784d...,1748255495,22566385,38.219307,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,38.175659,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 10:31:35+00:00
5,0xfdf67dc6d3e6c48b79c91f295646fd48837d06fbbf5c...,1748287895,22569065,28.657303,{'id': '0xc8eb2cf2f792f77af0cd9e203305a585e588...,28.631744,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xbA1462f43c6f60ebD1C62735c94E428aD073E01A,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0xC8Eb2Cf2f792F77AF0Cd9e203305a585E588179D,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-26 19:31:35+00:00
127,0xbc8853882a7d05247c9fb42f8a942fdab269b60f7fe6...,1748320307,22571756,21.137299,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,21.473808,{'id': '0x88794c65550deb6b4087b7552ecf29511379...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0x88794C65550DeB6b4087B7552eCf295113794410,2025-05-27 04:31:47+00:00
310,0x54ca0635165f7ec15b239fe39ae06e30be0932b9d9a2...,1748374523,22576246,44.264531,{'id': '0x6951bdc4734b9f7f3e1b74afebc670c736a0...,45.064088,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,0xe4433D00Cf48BFE0C672d9949F2cd2c008bffC04,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x6951bDC4734b9f7F3E1B74afeBC670c736A0EDB6,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2025-05-27 19:35:23+00:00


In [ ]:
def _connect_plans_to_rebalance_evnets(rebalance_event_df: pd.DataFrame, rebalance_plan_df: pd.DataFrame) -> dict:
    rebalance_transaction_hash_to_rebalance_plan = {}

    for index in range(len(rebalance_event_df)):
        one_rebalance_event = rebalance_event_df.iloc[index]

        tx_hash = one_rebalance_event["transactionHash"]
        same_destinations = (rebalance_plan_df["token_out"] == one_rebalance_event["tokenOutAddress"]) & (
            rebalance_plan_df["token_in"] == one_rebalance_event["tokenInAddress"]
        )
        same_amount_out = rebalance_plan_df["amount_out"] == one_rebalance_event["tokenOutAmount"]

        window_start = one_rebalance_event["datetime_executed"] - pd.Timedelta(minutes=10)

        generated_no_more_than_one_hour_before = rebalance_plan_df["datetime_generated"].between(
            window_start, one_rebalance_event["datetime_executed"]
        )

        matches = rebalance_plan_df[same_destinations & same_amount_out & generated_no_more_than_one_hour_before]
        matches = matches.sort_values("datetime_generated", ascending=False).head(1)

        if len(matches) == 0:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = None
        else:
            rebalance_transaction_hash_to_rebalance_plan[tx_hash] = matches["file_name"].values[0]

    return rebalance_transaction_hash_to_rebalance_plan


rebalance_transaction_hash_to_rebalance_plan = _connect_plans_to_rebalance_evnets(rebalance_event_df, rebalance_plan_df)

rebalance_event_df["rebalance_file_path"] = rebalance_event_df["transactionHash"].map(
    rebalance_transaction_hash_to_rebalance_plan
)
rebalance_event_df["autopool_vault_address"] = autopool.autopool_eth_addr
rebalance_event_df["chain_id"] = autopool.chain.chain_id


def _extract_rebalance_event_row(row: pd.Series) -> RebalanceEvents:
    return RebalanceEvents(
        tx_hash=row["transactionHash"],
        autopool_vault_address=row["autopool_vault_address"],
        chain_id=row["chain_id"],
        rebalance_file_path=row["rebalance_file_path"],
        destination_out=row["destinationOutAddress"],
        destination_in=row["destinationInAddress"],
        quantity_out=row["tokenOutAmount"],
        quantity_in=row["tokenInAmount"],
    )


new_rebalance_events_rows = rebalance_event_df.apply(_extract_rebalance_event_row, axis=1).to_list()

new_rebalance_events_rows

Index(['transactionHash', 'timestamp', 'blockNumber', 'tokenInAmount',
       'tokenIn', 'tokenOutAmount', 'tokenOut', 'destinationInAddress',
       'destinationOutAddress', 'tokenInAddress', 'tokenOutAddress',
       'datetime_executed', 'rebalance_file_path', 'autopool_vault_address',
       'chain_id'],
      dtype='object')

In [ ]:
# ensure that all of these transaction hashes are added

[RebalanceEvents(tx_hash='0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042e9753fe7c5e59e859b15', autopool_vault_address='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', chain_id=1, rebalance_file_path='rebalance_plan_1726602027_0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56.json', destination_out='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', destination_in='0xE382BBd32C4E202185762eA433278f4ED9E6151E', quantity_out=100.0, quantity_in=99.96101419887577),
 RebalanceEvents(tx_hash='0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8abf7eccc01e64cebb9dafc', autopool_vault_address='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', chain_id=1, rebalance_file_path='rebalance_plan_1726632022_0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56.json', destination_out='0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', destination_in='0xE382BBd32C4E202185762eA433278f4ED9E6151E', quantity_out=314.50037105465856, quantity_in=314.380623530208),
 RebalanceEvents(tx_hash='0xaa013ce865573532459a55487fb809c3e93edeed05af27ed14a4eb0b8284b4ae